In [1]:
import requests
import json
import re
import urllib3
import sqlite3 #数据库操作
import time
import random

/Users/wxy/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#定义保存评论的函数
def bcpl(weibo_id, base_url, headers, number):
    count = 1    #设置一个初始变量count为0来进行计数
    # with open("微博id" + str(weibo_id) + ".txt", "a", encoding="utf8") as f:    #打开一个名为“微博idxxxxxx”的txt文件，编码utf-8
    # 当count数量小于预期的number时，进行循环
    max_id = 4709743841249226
    max_id_type = 1
    while count < number:
        # 判断是不是第一组评论，如果是的话，第一组评论不需要加max_id，之后的需要加
        if count == 0:
            url = base_url + weibo_id + '&mid=' + weibo_id + '&max_id_type=0'
            print(url)
        else:
            url = base_url + weibo_id + '&mid=' + weibo_id + '&max_id=' + str(max_id) + '&max_id_type='+ str(max_id_type)
            print(url)
        try:  
            urllib3.disable_warnings()
            web_data = requests.get(url, headers=headers,verify=False)    #F12查看data信息
            js_con = web_data.json()    #转换一下数据格式
            # 获取连接下一页评论的max_id
            max_id = js_con['data']['max_id']  #max_id在[data]中
            max_id_type = js_con['data']['max_id_type']
            comments = js_con['data']['data']    #获得数据中[data]中的[data]
            comments_data_list = []
            for comment in comments:    #依次循环获得comments中的数据
                comments_data = []
                text = comment["text"]     #获得[text]下的数据，也就是评论数据
                label = re.compile(r'</?\w+[^>]*>', re.S)    #删除表情符号
                text = re.sub(label, '', text)    #获得文本评论
                created_at = comment["created_at"] #发布时间
                user = comment["user"]
                user_id = user["id"]
                user_name = user["screen_name"]
                #f.write(text + '\n')    #写入到文件中

                comments_data.append(text)
                comments_data.append(created_at)
                comments_data.append(user_id)
                comments_data.append(user_name)

                comments_data_list.append(comments_data)
                count += 1    #count = count + 1
                print("第" + str(count) + "条评论："+ user_name+" "+text  ) #显示爬取到第几条

            saveData2DB(comments_data_list,dbpath)
            time.sleep(random.uniform(2,7))
        except Exception as e:
            print("出错了" ,e)
            time.sleep(30)
            if(max_id_type==1):
                max_id_type = 0
            else:
                max_id_type = 1
            continue

In [3]:
def saveData2DB(datalist,dbpath):
    # init_db(dbpath)
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    sql = "insert into comments(text,created_at,user_id,user_name) values(?,?,?,?)"
    for data in datalist:
        cursor.execute(sql,data)
        conn.commit()
    cursor.close()
    conn.close()

In [4]:
def init_db(dbpath):
    sql = '''
        CREATE TABLE comments
        (
            id INTEGER  PRIMARY KEY AUTOINCREMENT,
            text varchar,
            created_at varchar,
            user_id varchar,
            user_name varchar
        )
    '''
    conn = sqlite3.connect(dbpath) #文件存在即打开，不存在即创建
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    conn.close()

In [5]:
dbpath = "J_SNOWMAN_4709336482318969.db"
#init_db(dbpath)
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9;',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'cookie':'WEIBOCN_FROM=1110006030; SCF=AmHexnmgkvxyMfOl7teccNNtI3I-aIptcc95t0_nN667Ax4MI-HcYc4Yn9rVZfwi7R5iVxR1cvE_3g-0S_Hyz4k.; SUB=_2A25MtmWvDeRhGeNH6lcV-SbLyzSIHXVsWQvnrDV6PUJbktAKLXnMkW1NSsw9B3oAwogWIs1su3XjfP2YaqPpC8F9; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhnV5Gg861gnb3kKqBEUXZD5NHD95Qf1K2fSh.RS05RWs4DqcjGxNHjIgSaTBtt; SSOLoginState=1639060991; _T_WM=39966695760; MLOGIN=1; XSRF-TOKEN=d39146; M_WEIBOCN_PARAMS=luicode%3D20000174%26uicode%3D20000174',
    'Referer': 'https://m.weibo.cn',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36',
    'Host': 'm.weibo.cn'
}
#设置user-agent来进行伪装，突破微博反爬限制
base_url = 'https://m.weibo.cn/comments/hotflow?id='
weibo_id = '4709336482318969'  # 要爬取的微博id  #[url=https://m.weibo.cn/detail/4478512314460101]https://m.weibo.cn/detail/4478512314460101[/url]
#打开微博手机端网页[url=https://m.weibo.cn]https://m.weibo.cn[/url]，找到要爬取的微博id！
#手机端网页！手机端网页！手机端网页！
number = 5000 # 设置爬取评论量,爬取量在第X组，爬取时会爬取下来该组的数据，所以最终数据可能会大于number，一般是个整10的数
bcpl(weibo_id, base_url, headers, number)

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709743841249226&max_id_type=1
出错了 'data'
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709743841249226&max_id_type=0
第2条评论：_柒夏 Love you both 
第3条评论：核桃的节操永远满格 有宣传消失的初恋诶！！！好耶！！！是因为这个剧认识的meme哦！！！如果可以！！！请一定要和道枝一起出演第二部！！！主题曲很好听！！！！专辑已经买啦！！！！！🖤🖤💖💖
第4条评论：莲理枝AI 中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第5条评论：Meguropanya 目黑莲！📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗📢活的好吗
第6条评论：陪你星程万里 消失的初恋超级好看，目黑大帅哥多来微博玩哦
第7条评论：柒鱼Senka 超喜欢！
第8条评论：从前有个养乐多 下次领着米七一起来微博📣📣📣
第9条评论：春日部防卫队-奶糕 期待消失的初恋第二季
第10条评论：W1BO-w 消失的初恋特别好看！！！！！宝 希望和我们michi二搭呀~！
第11条评论：本命cp目黑道枝 要是《消失的初恋》能拍成电影，或者有第二季就好了期待！！！
第12条评论：Phylliys 消失的初恋超级好看！歌也人超级好听！希望能多多看到meme和michi合作！消失的初恋第二季第二季！！专辑搞起来！[开学季][开学季][开学季]
第13条评论：杏菜打嗝 什么时候亲嘴啊
第14条评论：CAIWHOAMI 呜呜呜呜呜呜[开学季]消初好好看[春游家族][春游家族]谁把我一脚踹进日娱！？原来是meme和michi
第15条评论：小姜不早睡 meme晚上好呀！新歌真的好好听，已经等不及十二月的到来了呢！最近有好好吃饭嘛？你太瘦啦，要多吃点哦！有没有什么开心的事情呢？可以和我们分享一下

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=169504917309965&max_id_type=0
第122条评论：鹅美静儿 超级喜欢你演的井田！！消失的初恋真的好好看！帅哥们可以再次合作演消失的初恋第二季吗
第123条评论：解雨戏臣 都让让都让让，井田和青木的婚车🚚要来了🥰🥰都让让都让让，井田和青木的婚车🚚要来了🥰🥰都让让都让让，井田和青木的婚车🚚要来了🥰🥰都让让都让让，井田和青木的婚车🚚要来了🥰🥰都让让都让让，井田和青木的婚车🚚要来了🥰🥰
第124条评论：敲哩喵屁屁 消失的初恋井田君演的超级棒 每周都有在TVer看 五单也买了 secret touch非常好听
第125条评论：易长屿 爱你俩meme消初第二季gkd
第126条评论：咪酱-咩黒蓮 宝好努力好感动，中文进步了！
第127条评论：幸好不是他啊 我们鸟人和你俩一起共创未来[春游家族][春游家族][春游家族]赶紧安排新的合作[春游家族][春游家族][春游家族]杰尼斯爸爸求求了🙊🙊🙊
第128条评论：33等zjl新歌 meme看起来你最近活的很好[赢牛奶]🖤🖤中文又进步了🖤🖤[赢牛奶][赢牛奶][赢牛奶]《secret touch》已经预订🖤🖤 [开学季][开学季][开学季]超喜欢《消失的初恋》🖤🖤
第129条评论：Menanaisen 有在看你演的电视剧啊 我就是因为这部电视剧喜欢你的 很高兴通过这部电视剧认识你 希望未来你可以多拍电视剧和电影作品 期待你的表现啊[赢牛奶][赢牛奶][赢牛奶]
第130条评论：璇若kid 消失的初恋第二季！
第131条评论：熙熙甜豆豆 Meme真的很棒，所以第二季懂吗
第132条评论：熙熙甜豆豆 消初很甜消初很好看meme特别棒，所以求求第二季消初很甜消初很好看meme特别棒，所以求求第二季消初很甜消初很好看meme特别棒，所以求求第二季消初很甜消初很好看meme特别棒，所以求求第二季消初很甜消初很好看meme特别棒，所以求求第二季消初很甜消初很好看meme特别棒，所以求求第二季
第133条评论：薄湘 我上辈子一定是周敦颐这辈子才这么爱你
第134条评论：肖samyu阳 第二季消初搞快点!
第135条评论：Mark93_Tuan 祝贺五单取得好成

第242条评论：壹时行时不行 我看到这些已经毫无波澜了，已经伤不到我了，我在大润发杀了十年的鱼，我的心早已跟我的刀一样冷了，我还在少林寺扫了八年落叶，我的心早已和风一样凉了，我还在长江游了十年冬泳，我的心早已和水一样冰了。。目黑莲。。俺家的高粱都知道俺稀罕你！
第243条评论：芝士葡萄玛奇朵 真的希望能有第二季呀mememichi的合作太棒了！
第244条评论：请INTO1保护好der妹 消失的初恋第二季
第245条评论：·宇宙夹在书页 有看有看！《消失的初恋》超级好看！新歌也很好听！
第246条评论：二月紺香 莲酱你来啦！！！！你讲中文的样子也很可爱好想念你呀！！五单已经购入了，就等着收到了，很喜欢secret touch这首歌！！meme酱唱抒情歌曲也很棒！歌曲开头部分真的让我很想落泪（因为太好听了）以后也在中国见面吧，很想念你，注意身体，期待meme未来更好的表现🖤
第247条评论：柠栀纳米 meme晚上好！超级期待五单 真的好喜欢你 要好好照顾自己 期待和你在中国相见🖤
第248条评论：二缺没有心i 很喜欢！！我们真的很期待消失的初恋第二季！！！请一定要和道枝一起出演第二部！！！[开学季]
第249条评论：敲哩喵屁屁 新单曲买了买了买了买了买了 没有消失的初恋我可怎么活啊 就剩两集了[苦涩]
第250条评论：爱吃西米露的狼 放心，我们都有干消失的猪恋
第251条评论：须臾鹅一定会运气爆表 希望你们能够一直发糖！！！！
第252条评论：米七七ya 目黑莲下次带着道枝骏佑一起同框！带着道枝一起！带着道枝一起！《消失的初恋》希望能拍第二季！！！
第253条评论：咔叽呀 和道枝结婚吧
第254条评论：KryoYee 喜欢消失的初恋
第255条评论：是Miku吖 第二部，第二部！！！
第256条评论：-Olayinka- 有在看消失的初恋哦！井田非常非常温柔呢！！🖤🖤🖤
第257条评论：g皓哥哥欧 老公老公啊啊啊啊，啵啵(^з^)，要找你啵啵(^з^)
第258条评论：是Miku吖 meme你的心我们get到啦！！
第259条评论：全糖小峰不加冰 快拍第二季！！！
第260条评论：karry20030312 看了看了，secret touch爱了，超级好听的，你和道枝好配啊。
第261条评论：咻咻xi 消失的初恋非常好看！希望有第二季❤️
https://m.weib

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4712185055939679&max_id_type=1
第408条评论：不加糖的Chris 感谢有消失的初恋这部剧，认识了meme和michi， 进而又认识了雪人全团和南泥湾全团，以后也会继续支持两个团的另外剧真的太好看啦，跪求第二季第二季第二季！！！Second Season
第409条评论：说来说去都是话 希望消失的初恋能够有吻戏🙏，献上一张网友P的图片
第410条评论：信号一直在 已经吃土在攒钱准备买box
第411条评论：陆詟水栗sama 二搭二搭二搭
第412条评论：阁77564 看了看了我很喜欢
第413条评论：阿泫今天脱非入欧了吗 消失的初恋第二季！！！一定要有啊
第414条评论：0号鸭子 血书第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季
第415条评论：慭律 第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季目黑莲想要第二季
第416条评论：奈斯泡芙7 消初能不能先拍个20季的！！！
第417条评论：许我风尘 我也想要第二季，割我韭菜请求
第418条评论：-燕勒- 消失的初恋第二季！
第419条评论：橙子味奶啤 meme自己在采访中说了想拍第二部，所以快来吧
第420条评论：仙女的酒窝是戳出来的 求求了！
第421条评论：仙女的酒窝是戳出来的 第二季！
第422条评论：假的GTC 第二部！
第423条评论：丧心病狂的糯米粽子 第二季第二季啊！meme也想拍续集啊
第424条评论：六祘榖 钱包已经准备好了，赶快安排一下第二季！！
第425条评论：·宇宙夹在书页 他们俩想拍！我们想看！求求了第二季🙏
第426条评论：·宇宙夹在书页 二搭或者第二季总要来一个吧🙏
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4712167376686217&max_id_type=1
第427条评论：0号鸭子 第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季第二季[赢牛奶][赢牛奶]
第42

第602条评论：水果糖精yes 有没有皇家翻译 翻译一下
第603条评论：沼伞暮泗 看来你的中文确实说的比英文好哈哈哈哈哈哈
第604条评论：沼伞暮泗 消失的初恋好好看！！！
第605条评论：Ginko酱 消失的🐷恋
第606条评论：哎哟喂93161 一直在追呢一直在追呢
第607条评论：michinokanojo 有 meme中文讲得真好 都听懂了
第608条评论：昂帕帕斯415 超级喜欢
第609条评论：西瓜怪兽biu 超过150万了耶
第610条评论：佑小夕 meme你讲中文很可爱
第611条评论：纸鸢物语 目黒くんわざわざ動画上げてくれてありがとうございます！中国のみんなも井田くんのこと、青木くんのこと大好きだから、いつかまたお二人に会いたいと願っております！🙏笑　最近みんな忙しくて中国語のお勉強できてるか少し心配で、敢えて日本語で書きました笑　これからも中国語頑張ってください！
第612条评论：青菜粥配鱼糕 晚上好！！！
第613条评论：苏知甜 meme
第614条评论：theanothertwo 每天来看meme
第615条评论：是蕾蕾本尊小姐姐 厉害哟
第616条评论：宇智波鼬官方女友 太可爱了宝
第617条评论：橘橘数据2号 
第618条评论：Fairikkkkkk-丸子 meme 什么时候来中国！！
第619条评论：橘橘数据5号 
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4711415556345379&max_id_type=1
第620条评论：zhiaiblt 
第621条评论：喝杯橘子水- 井田&amp;青木kiss📢井田&amp;青木kiss📢井田&amp;青木kiss📢井田&amp;青木kiss📢井田&amp;青木kiss📢井田&amp;青木kiss📢
第622条评论：Seay_dz 你活的好吗你活的好吗你活的好吗你活好吗
第623条评论：ee遇景行 为什么微博会自动掉赞meme！meme！meme！meme！
第624条评论：夜间飞行浪漫号 12月浪漫水瓶，怎么过？
第625条评论：买了衣服都得穿 哦莫原来有微博，我又在lb追星
第626条评论：柚西yoka 
第627条评论：田綵雲 更新好慢咱就是说有没有第二

...

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709371122553510&max_id_type=1
第4226条评论：SY贺先 期待meme和Micchi的更多合作呀！帅哥要多多营业！🖤💖  网页链接
第4227条评论：小树子_Fancy 消失的初恋第二季拜托了🙏🙏
第4228条评论：萧仙儿QAQ 请一定要和道枝一起出演消失的初恋第二季啊
第4229条评论：风太大我听得见 今年冬天，我的体寒都好了！温暖是消失的初恋带给我的！请一直暖下去好吗  网页链接
第4230条评论：A爱丽丝和6529177770 看了看了，汉化更新好慢
第4231条评论：月下千聲 meme超ーーーーーーーかっこいい！！！「消えた初恋」もめっちゃ素晴らしいドラマです！！！中国のファンはmemeもmicchiも好きすぎて、このドラマはまだ終わらないが、もう続編楽しみにしています
第4232条评论：半夜弹钢琴来吓你 ！！！！！！！！超级好看啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊！！就是因为这个才入坑了希望会有第二季啊啊啊啊meme辛苦了[开学季]
第4233条评论：番茄煮万物 我！活！了！ 好的可以睡一个甜甜的觉了
第4234条评论：今天开心点儿叭 消失的初恋第二季我很期待[开学季][开学季][开学季]
第4235条评论：糖果专用 转发微博
第4236条评论：敦子女士 超喜欢超喜欢你和michi都超级好看！！！！
第4237条评论：仓鼠侠的饼干妈妈 micchimeme is the best!!!
第4238条评论：求个机会为人民服务 咩咕噜中文很行，我很爱也爱米七
第4239条评论：宝贝女鹅七塔蹦 我好喜欢你啊meme
第4240条评论：南起酥糖 meme晚安Snow Man晚安
第4241条评论：我是你的小阿明呐 memememe！！！零国语言能力者今天中文说的有点子好耶🙊🙉🙈🈶️偷偷加练耶
第4242条评论：Noya敲可爱 超级喜欢你和michi哦
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709370871680976&m

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709369478121658&max_id_type=1
第4391条评论：桃色案件启示录 啊啊啊啊啊啊啊啊啊啊啊
第4392条评论：胖团子是我哒 第二季第二季第二季
第4393条评论：布谷鸟布谷谷布 maguro桑
第4394条评论：社畜只想睡觉 看了 买了 很喜欢哈哈哈哈哈哈哈哈 说中文怎么会这么有趣呢meme
第4395条评论：未病_ 求求消えた初恋2️⃣！！电影也可以！！二搭也可以，只要是你俩就都行🖤💗
第4396条评论：雨雾小谢 正在想您和米七（^^）
第4397条评论：土豆和鸡蛋不可兼得 带米七一起来中国玩啊，消失的初恋第二季等你们
第4398条评论：花开不开都无所谓了 消失的初恋超好看
第4399条评论：5号橘子呀 @0808__ @揍你嗷 兄弟们快帮我给这条微博点个赞！
第4400条评论：再别烦我r 立马写一首《爱 莲 说》
第4401条评论：卷禾里 都很喜欢哦
第4402条评论：嗑尽天下西皮 是爱情吧!是爱情吧!啊啊啊啊啊啊啊
第4403条评论：ProtegoTotalum [春游家族][春游家族][春游家族]消失的初恋！请速速拍电影！两位神仙请速速二次合作！📣📣📣
第4404条评论：Lovelykx 看了的，期待消失的初恋第二季💗[春游家族]
第4405条评论：玛卡巴卡不自扰 消失的初恋第二季期待期待期待
第4406条评论：丶保持安静 消失的初恋第二季，请安排上啊。
第4407条评论：飞了飞了104 全中文
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709369352028643&max_id_type=1
第4408条评论：memi和比翼鸟 你和michi会拍消失的初恋第二季吗？
第4409条评论：万能人永远爱ONER 消失的初恋太好看啦！！！井田这个角色诠释的特别棒 希望能拍第二季 期待你和道枝再次合作！！！！我们都特别想看！！！！（消えた初恋イケすぎ! ! !あなたは井田のこの役の解釈の特別なすばらしいです! ! !道枝くんとはまたシーズン2で一

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709368060448322&max_id_type=1
第4573条评论：Berbernic めめ🖤🖤🖤
第4574条评论：是罐装番茄吖 meme
第4575条评论：幕远行 meme
第4576条评论：Lolli-泡泡 meme爱你
第4577条评论：SCKDXBL 目黑莲！活的好吗？托你的福我快笑死了🤣，有在好好看剧五单也会买，第一次追日娱真的好麻烦
第4578条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4579条评论：Sunset巴士站 
第4580条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4581条评论：解雨戏臣 真的太帅气了
第4582条评论：落雁堂前 大莲哥 你好帅哦 消失的初恋很好看 请莫多莫多 蓮くん　がっこいい！　消えた初恋がとても好き！もっと見たい
第4583条评论：X阿拉蕾L 快点来中国开演唱会吧
第4584条评论：eternalw- memeはかっこいいです。secret touchは綺麗です。『消えた初恋』の第二シーズンを楽しみにしています。memeとmicchiを楽しみにしています。🖤💗🖤💗🖤💗@J_SnowMan
第4585条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4586条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4587条评论：Bipolar_x 宝！！我已经买了secret touch了！！！你拍的剧很好看！希望你拍更多的剧！
第4588条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4589条评论：不愿再痛我想下楼 谢谢你很用心准备了中文💗❤️话说《消失的初恋》能不能拍第二季呢！在中国真的超级有人气的说！超级喜欢！！
第4590

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709366860878362&max_id_type=1
第4720条评论：草莓啵啵冰激凌 啊啊啊啊啊，我的宝儿
第4721条评论：嗑尽天下西皮 二搭搞起来！
第4722条评论：糊涂小仙分仙 评论过万了！给力啊
第4723条评论：啊呀呀死瘦子 太厉害了吧，啥时候第二季
第4724条评论：冲田抱熊 中国語で話してくれてありがとう！！！secret touchもう全バージョン予約したよ！！すっごくいい曲で毎日聞いてるよ！！！このトラムでmemeを知って、今はもうすっかりsnow manのファン，ここまで頑張ってくれてありがとう！！私にsnow manを知るチャンスをくれてありがとう！！
第4725条评论：木西酱_cc meme🖤🖤🖤多吃点吧宝
第4726条评论：我还是个孩子啊嘤 咱就说不管是唯粉还是cpf在前排都记得买五单就行了嗷
第4727条评论：一事无成熬夜大师 睡得晚就会有好事
第4728条评论：爱吃糖的榴莲 哇，meme中文可以啊，虽然发音很用力，但是能够用这么一整段的中文发言，厉害了！
第4729条评论：琳琅xx 消えた初恋📢2⃣！消えた初恋📢2⃣！消えた初恋2⃣！消えた初恋📢2⃣！消えた初恋📢2⃣！消えた初恋📢2⃣！🖤💗  网页链接
第4730条评论：Famas_ 消失的初恋很好看，每周都很期待！！希望能有第二季呢！SnowMan的secret touch也超级好听!!!
第4731条评论：壹岁朝 你的中文学的很好，加油！！
第4732条评论：·双林林林林林 会一直支持SnowMan的!!!宝贝有在努力说中文呢!辛苦了!!!!
第4733条评论：麦克列yu 《消失的初恋》我们都有看哦！大家都很期待第二季呢！《secret touch》也非常好听！祝五单大麦！欢迎meme和michi来中国玩！也期待snow man来中国开演唱会！  网页链接
第4734条评论：嗯是冰雪没错了 呜呜呜呜呜呜呜大莲哥！好喜欢看你和michi的消失的初恋！！！🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗🖤💗
第4735条评论：恒温怪怪 要第二季要第二季要第二季要第二季要第二季要第二季要第二季要

https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709365842186196&max_id_type=1
第4887条评论：森中一绯 我笑的像踩中地雷哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈这个中文
第4888条评论：小洁癖2020 多点来weibo玩meme
第4889条评论：我不是一张饼 在看了在看了，很喜欢
第4890条评论：Boboboninini 第二季第二季第二季！！！！中国的大家都很喜欢你们
第4891条评论：Ghsasjh 消失的初恋gkd
第4892条评论：大脸怪阿狸 消失的初恋第二季，或者电影，强烈希望二搭呀！
第4893条评论：不想做报表 Aaaa
第4894条评论：-雅lara 啊啊啊啊啊啊爱你
第4895条评论：-兔兔面包 @搞只猫来
第4896条评论：猫夜_今天为山田家打碟了吗 📣有在看——————
第4897条评论：是我在熬夜_ 消えた初恋！消えた初恋！消えた初恋！消えた初恋！消えた初恋！消えた初恋！超级喜欢你们🖤💗想要第二季！！！！！
第4898条评论：简单咖啡- meme
第4899条评论：wuli抽抽意密达 消除实🔥！！
第4900条评论：三重Ryoma 听不懂宝 但是没关系 已经all了两套了
第4901条评论：Yoomi小酸奶_ memememememememememe
第4902条评论：楼谢莉 meme=咩咩🐑
第4903条评论：芝士星球星冰乐 中国語を話すのがとてもかわいいですね~memeとmichi二人は本当に素晴らしいです！！！！頑張れね!!
第4904条评论：壹岁朝 希望你们一直拍下去！！！
第4905条评论：wuli抽抽意密达 鸟人们给我 冲！！！
第4906条评论：0号鸭子 //@从前有个养乐多:消初第二季gkd📣
https://m.weibo.cn/comments/hotflow?id=4709336482318969&mid=4709336482318969&max_id=4709365699839171&max_id_type=1
第4907条评论：肆百擊中花瓣了 超级爱你啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊

In [ ]:
&max_id=4709743841249226&max_id_type=1